In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import dgl
import torch
import torch.nn as nn
import itertools
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from dgl.nn import SAGEConv
import dgl.function as fn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder

## Рекомендательная система

Чтобы удобнее было использовать рекомендательную систему, спарсим названия компаний по их ИНН.

In [ ]:
companies = pd.DataFrame(set(final_dataset['ИНН поставщика']) | set(final_dataset['ИНН заказчика']), columns = ['ИНН'])
companies

,ИНН
0,6670381056
1,3234013184
2,1644040195
3,2536038419
4,4217102358
...,...
1018,7707279342
1019,3128115184
1020,7805018099
1021,7709007859


In [ ]:
def parsing_name_of_company(num_tin, df_tin):
    url_tin = f'https://companies.rbc.ru/search/?query={num_tin}'
    ua = UserAgent(browsers=['chrome'])
    headers = {"User-Agent": ua.random}

    response_tin = requests.get(url_tin, headers=headers)
    soup = BeautifulSoup(response_tin.text, "lxml")

    name = None
    if soup.find("a", class_="company-name-highlight") != None:
        name = soup.find("a", class_="company-name-highlight").text

    dict_data = pd.DataFrame({'ИНН': [num_tin], 'Название компании': [name]})

    df_tin = pd.concat([df_tin, dict_data], ignore_index=True)

    return df_tin

In [ ]:
df_companies = pd.DataFrame(columns=['ИНН', 'Название компании'])

for num_tin in tqdm(companies['ИНН'].unique()):
    df_companies = parsing_name_of_company(num_tin, df_companies)
    sleep(0.0001)

100%|██████████████████████████████████████████████████████████████████████████████| 1023/1023 [05:05<00:00,  3.35it/s]


**Время**, затраченное на парсинг названий компаний: 5 мин. 5 сек.

In [ ]:
df_companies

,ИНН,Название компании
0,6670381056,"ООО ""ЕКАТЕРИНБУРГ ЯБЛОКО"""
1,3234013184,"ГАУЗ ""БООД"""
2,1644040195,"ООО ""ТАТНЕФТЬ-АЗС ЦЕНТР"""
3,2536038419,"КГА ПОУ ""ЭНЕРГЕТИЧЕСКИЙ КОЛЛЕДЖ"""
4,4217102358,"АО ""НХС"""
...,...,...
1018,7707279342,"ООО ""МЕРСЕДЕС-БЕНЦ ФАЙНЕНШЛ СЕРВИСЕС РУС"""
1019,3128115184,"МАУ ТЦ ""ТЕНХАУС"""
1020,7805018099,"ООО ""ГАЗПРОМ ТРАНСГАЗ САНКТ-ПЕТЕРБУРГ"""
1021,7709007859,"ФГУП ""ФТ-ЦЕНТР"""


Построим функцию, принимающую на вход все параметры, которые можно добавить как фичи, но параметры, определяющие фичи исполнителей контрактов, будут по умолчанию 0, отношение к классу "заказчик" - единицей, а количество рекомендованных имполнителей - 1.

In [ ]:
def recommendation(price, time, okpd2, сount = 1, supplier = 0,
                   purchaser = 1, revenue = 0, growth_rate = 0, authorized_capital=0,
                   df_nodes = df_nodes, df_companies = df_companies):

    # Создаем фичи для вершины графа
    node_feat = np.array([authorized_capital, revenue, growth_rate,
                          price, price, price, price, price,
                          time, time, time, time, time,
                          supplier, purchaser])

    okpd2_f = onehotencoder.transform(pd.DataFrame(data=[okpd2]))
    okpd2_f = pd.DataFrame(okpd2_f.toarray(), columns=onehotencoder.categories_[0])
    node_feat = np.append(node_feat, okpd2_f)


    node_mean=np.array(df_nodes.drop(['ИНН', 'Ребра'], axis=1).mean())
    node_std=np.array(df_nodes.drop(['ИНН', 'Ребра'], axis=1).std())

    node_feat = (node_feat - node_mean) / node_std
    node_feat = node_feat.astype(np.float32)
    node_feat = torch.from_numpy(node_feat)
    node_feat = node_feat.unsqueeze(0)

    # Строим граф
    RG = nx.DiGraph()

    RG.add_nodes_from(df_nodes['ИНН'])

    for i in range(len(df_nodes)):
        RG.nodes[df_nodes['ИНН'][i]]['feat'] = features[i]

    rg = dgl.from_networkx(RG, node_attrs=['feat'])

    rg.add_nodes(1, {'feat': node_feat})

    # Добавляем новые ребра
    new_edges = []

    if supplier:
        for i in range(rg.number_of_nodes() - 1):
            new_edges.append((i, rg.number_of_nodes() - 1))

    if purchaser:
        for i in range(rg.number_of_nodes() - 1):
            new_edges.append((rg.number_of_nodes() - 1, i))

    new_edges = np.array(new_edges)
    rg.add_edges(new_edges[:, 0], new_edges[:, 1])

    # Применяем нашу модель
    h = model2(rg, rg.ndata['feat'])
    pos_score = pred2(rg, h)
    pos_score = pos_score.detach().numpy()

    сount = np.argsort(pos_score)[-сount:]
    сount = сount[::-1]

    сount_TIN = []
    count_companies = []
    for i in сount:
        сount_TIN.append(df_nodes['ИНН'][i])
        count_companies.append(df_companies['Название компании'][i])
    print(f'Для контракта с ценой {price} руб., временем выполнения {time} дней(дня) и ОКПД2 = {okpd2} Вам рекомендованы следующие исполнители: \n\
{count_companies} (ИНН: {сount_TIN})')

In [ ]:
recommendation(price = 1_000_000, time = 3, okpd2 = '72')

Для контракта с ценой 1000000 руб., временем выполнения 3 дней(дня) и ОКПД2 = 72 Вам рекомендованы следующие исполнители: 
['ФГУП "РОСМОРПОРТ"'] (ИНН: [7702352454])


In [ ]:
df_nodes[df_nodes['ИНН'] == 7702352454].iloc[:, :17]

,ИНН,Ребра,Уставной капитал,Выручка,Темп прироста %,Цена sum,Цена mean,Цена max,Цена min,Цена median,Время sum,Время mean,Время max,Время min,Время median,Поставщик,Заказчик
141,7702352454,[],1.212243e+11,3.442041e+10,20.7,79009409.15,4.938088e+06,29700000.0,0.11,499999.0,63.0,3.9375,71.0,-25.0,0.0,1.0,0.0


In [ ]:
recommendation(price = 1_000_000, time = 3, okpd2 = '72', сount = 3, supplier = 1, purchaser = 0)

Для контракта с ценой 1000000 руб., временем выполнения 3 дней(дня) и ОКПД2 = 72 Вам рекомендованы следующие исполнители: 
['БУЗОО "КПБ ИМ. Н.Н. СОЛОДНИКОВА"', 'АМУ "КДЦ "ЮЖНЫЙ"', 'ГБУ РО "СП" В АКСАЙСКОМ РАЙОНЕ'] (ИНН: [5504004973, 4703077727, 6102005111])


In [ ]:
df_nodes[(df_nodes['ИНН'] == 5504004973) | (df_nodes['ИНН'] == 4703077727) | (df_nodes['ИНН'] == 6102005111)].iloc[:, :17]

,ИНН,Ребра,Уставной капитал,Выручка,Темп прироста %,Цена sum,Цена mean,Цена max,Цена min,Цена median,Время sum,Время mean,Время max,Время min,Время median,Поставщик,Заказчик
213,5504004973,[7709678550],0.0,0.0,0.0,195990.0,195990.0,195990.0,195990.0,195990.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
354,4703077727,[7804069580],0.0,0.0,0.0,623023.3,623023.3,623023.3,623023.3,623023.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
363,6102005111,[6168002922],0.0,0.0,0.0,822432.0,822432.0,822432.0,822432.0,822432.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


Таким образом, мы сделали рекомендательную систему для рекомендации исполнителей с учетом цены и времени исполнения контракта.